# Import von WIAG-Daten in FactGrid
Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [1]:
using CSV, DataFrames, Dates

In [10]:
ENV["COLUMNS"] = 120

120

In [2]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [3]:
output_path = input_path

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [48]:
input_file = "WIAG-can_2023-12-05_lokal.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-can_2023-12-05_lokal.csv"

In [49]:
df_person = CSV.read(input_path_file, DataFrame);
nrow(df_person)

15136

Das deutsch Beschreibungsfeld enthält Lebensdaten mit der Zusammenfassung der Amtsdaten

In [50]:
join_komma(a, b) = join((a, b), ", ")
transform!(df_person, [:biographical_dates, :summary_offices] => ByRow(join_komma) => :description_de);

In [51]:
df_person[8:10, [:givenname, :prefix, :familyname, :description_de]]

,givenname,prefix,familyname,description_de
,String,String31?,String?,String
1,Ulrich,von,Eichheim,"+ 1324, Domherr Domstift Würzburg vor 1324, Dompropst Domstift Augsburg 1321-1324"
2,Eberhard,von,Öttingen,"~ 1324/1331, Propst und Archidiakon Benediktinerkloster Ansbach, später Kollegiatstift 1331, Propst Kollegiatstift St. Maria, Feuchtwangen 1331"
3,Marquard,von,Randeck,"+ 1381, Domherr Domstift Augsburg vor 1381, Patriarch Aquileia 1365-1381"


Benenne die Spalten um entsprechend den Konventionen des FactGrid

In [52]:
columns = [
    :displayname => :Lde,
    :description_de => :Dde,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601,
    :Wikidata_ID => :Swikidatawiki,
    :Wikipedia => :Sdewiki
];

In [54]:
rename!(df_person, columns);

Kopiere das Label in Deutsch auch als Label in Englisch, Französisch, Spanisch

In [55]:
df_person.Len = df_person.Lde;
df_person.Lfr = df_person.Lde;
df_person.Les = df_person.Lde;

Kopiere die Lebensdaten als Beschreibung für Englisch, Französisch, Spanisch

In [56]:
df_person.Den = df_person.biographical_dates;
df_person.Dfr = df_person.biographical_dates;
df_person.Des = df_person.biographical_dates;

Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [57]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [43]:
fqs_string(s) = "\"" * string(s) * "\""

fqs_string (generic function with 1 method)

In [44]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

create (generic function with 1 method)

In [45]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

set_property_list

Gib ausgewählte Elemente aus `df_person` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [34]:
date_key = Dates.format(now(), "yyyy-mm-dd")

"2023-12-05"

In [60]:
output_file = "Insert_WIAG-can_lokal_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\Insert_WIAG-can_lokal_2023-12-05_FQ.csv"

In [61]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_person)
        create(out_stream, row, [:Lde, :Len, :Lfr, :Les, :Dde, :Den, :Dfr, :Des])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472, :Swikidatawiki, :Sdewiki], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string])
    end        
end